# Training file (testing if training works)

In [2]:
import sys
sys.path.append('../')

In [3]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.nn import CTCLoss
import torch.optim as optim

from src.training.trainer import train_model
from src.dataset.custom_dataset import OdometerDataset
from src.dataset.base_dataset import base_collate_fn
from src.models.crnn import CRNN

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [ ]:

data_dir = '/home/yannou/OneDrive/Documents/3_PRO/carviz/data/odometer_reader/milestone_box'
labels_file = '/home/yannou/OneDrive/Documents/3_PRO/carviz/data/odometer_reader/milestone_box/milestone_labels.json'

# Définir les transformations
transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

transform_val = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Initialiser le dataset
dataset = OdometerDataset(root_dir=data_dir, split="train", labels_file=labels_file, img_height=32, img_width=100, transform=transform)
dataset_val = OdometerDataset(root_dir=data_dir, split="val", labels_file=labels_file, img_height=32, img_width=100, transform=transform_val)


# Créer les DataLoaders
train_loader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=base_collate_fn)
valid_loader = DataLoader(dataset_val, batch_size=64, shuffle=False, collate_fn=base_collate_fn)



In [7]:
# Initialiser le modèle
num_class = len(OdometerDataset.LABEL2CHAR) + 1
crnn = CRNN(img_channel=1, img_height=32, img_width=100, num_class=num_class, model_size="xl", leaky_relu=True).to(device)

# Définir les paramètres d'entraînement
lr = 0.001
epochs = 2
decode_method = 'beam_search'
beam_size = 10
label2char = OdometerDataset.LABEL2CHAR

In [8]:
# Appeler la fonction train_model
trained_model = train_model(
    model=crnn,
    train_loader=train_loader,
    valid_loader=valid_loader,
    label2char=label2char,
    device=device,
    lr=lr,
    epochs=epochs,
    decode_method=decode_method,
    beam_size=beam_size,
    criterion=CTCLoss(reduction='sum', zero_infinity=True).to(device),
    optimizer=optim.RMSprop(crnn.parameters(), lr=lr),
    project_name="test",
    run_name="test"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yann-t (carviz-com). Use `wandb login --relogin` to force relogin


Epochs:  50%|█████     | 1/2 [03:14<03:14, 194.68s/it]

Epoch 1: train_loss=16.251576704908704, train_accuracy=0.0, val_loss=13.582022099721256, val_accuracy=0.0


Epochs: 100%|██████████| 2/2 [06:19<00:00, 189.65s/it]

Epoch 2: train_loss=13.594267849836333, train_accuracy=0.0, val_loss=13.558009370844415, val_accuracy=0.0


RuntimeError: Parent directory ./models does not exist.